### 할로윈 투자 전략
1. 11월 첫날에 주식을 매수 -> 6개월간 유지 4월 말일에 판매
2. 11월 첫날의 시가로 구매
3. 4월 마지막날의 종가로 판매
4. 수익율 -> (3번과정 - 2번과정) / 2번과정 + 1
5. 주식 데이터를 가지고 수익율 계산

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
df = pd.read_csv("../../csv/AMZN.csv", index_col='Date')
df.head(1)

In [ ]:
# index를 시계열 변경
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

In [ ]:
# 2000년 11월 첫날의 데이터 출력(시가)
df.loc['2000-11'].iloc[[0]]

In [ ]:
# 2001년 4월 마지막 날의 데이터 출력(종가)
df.loc['2001-04'].iloc[-1]['Close']

In [ ]:
# 2000년부터 2019년까지 11월의 첫날의 데이터와 4월의 마지막날의 데이터만 추출하여
# 새로운 데이터프레임을 생성

# 1. 비어있는 데이터프레임 생성
price_df = pd.DataFrame()
# 2. 반복문을 이용하여 2000년부터 2019년까지 반복
for i in range(2000, 2019):
    # 11월 첫날의 데이터를 추출
    buy_mon = str(i) + "-11"    # 2000-11
    buy = df.loc[buy_mon].iloc[[0]]
    # 데이터프레임에 결합
    price_df = pd.concat([price_df, buy], axis=0)
    # 다음 년도의 4월 마지막의 데이터를 추출
    sell_mon = str(i+1) + '-04' # 2001-04
    sell = df.loc[sell_mon].iloc[[-1]]
    # 데이터 프레임에 결합
    price_df = pd.concat([price_df, sell], axis=0)

price_df


In [ ]:
# 시가와 종가 컬럼만 추출
price_df = price_df[['Open', 'Close']]
price_df.head(2)

In [ ]:
# rtn 컬럼을 생성 데이터는 1
price_df['rtn'] = 1
price_df.head(2)
# 4월 데이터에서 수익율 발생 -> 전행의 시가와 현재행의 종가로 수익율 발생

In [ ]:
# 인덱스가 4월인 조건식?
price_df.index.month == 4

In [ ]:
for i in price_df.index:
    # i는? -> index
    # index가 4월인 경우
    if i.month == 4:
        # 수익율을 계산 (현재행의 종가 - 전행의 시가) / 전행의 시가 + 1
        rtn = (price_df.loc[i, 'Close'] - price_df.shift(1).loc[i, 'Open']) \
        / price_df.shift(1).loc[i, 'Open'] + 1
        # 수익율을 rtn에 대입 
        price_df.loc[i, 'rtn'] = rtn

price_df.head(6)


In [ ]:
# 누적 수익율 계산
acc_rtn = 1
for i in price_df.index:
    rtn = price_df.loc[i, 'rtn']
    acc_rtn *= rtn

acc_rtn

In [ ]:
from dateutil.relativedelta import relativedelta

In [ ]:
# 2000년 11월 1일 시계열 데이터를 생성
start = datetime(year=2000, month=11, day=1)
start

In [ ]:
# 매도 월 
# 시작 시간에서 5개월 뒤 
end  = start + relativedelta(months=5)
end

In [ ]:
print(start.strftime('%Y-%m'), end.strftime('%Y-%m'))

In [ ]:
def six_month(df, start = 2000, end = 2019, month = 11):
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True)
    # 인덱스를 시계열로 변경
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

    # 누적수익율
    acc_rtn = 1

    for i in range(start, end):
        _start = datetime(year = i, month = month, day = 1)
        _end = _start + relativedelta(months=5)

        # 구매하는 달
        buy_mon = _start.strftime('%Y-%m')
        # 판매하는 달
        sell_mon = _end.strftime('%Y-%m')

        # 구매한 가격 
        buy = df.loc[buy_mon].iloc[0]['Open']
        # 판매한 가격
        sell = df.loc[sell_mon].iloc[-1]['Close']

        # rtn = (sell - buy) / buy + 1
        rtn = sell / buy

        # 누적수익율 계산
        acc_rtn *= rtn
        
    return acc_rtn

In [ ]:
six_month(df)

In [ ]:
# 1월부터 12월까지 반복
for i in range(1, 13):
    print(f"{i}월의 누적 수익율 :", six_month(df, month=i))

In [ ]:
df2 = pd.read_csv("../../csv/AAPL.csv", index_col='Date')

In [ ]:
for i in range(1, 13):
    print(f"{i}월의 누적 수익율 :", six_month(df2, month=i))

In [51]:
import halloween

In [52]:
halloween.six_month(df2)

8.177103601339539